In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'nlp-comp:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4733852%2F8031300%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T090221Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D73f8c18607d33bec368520e685d4a082c72cd325c528c243a26c69a08da1ed8f343cdada8ba3f098277d5256fddc04a7c8d80f9305308a33c7174f7bdf8df082d0a65504d7e104119ca5a3c9a1f5fc58c0d985d9e2b8da6546046e56dadd2db295ef2338c36667e7f9e23c466fce91da861547a9d1e0756822f4cea5073f7937184ad70d16a60aa4d1fdae1a1f15452b6e3c28459c1f010a8b0d7377efef6dcf5e55ebb886770bfcf2e4545ca039fe2b9a827b8c316173a1419bb163b590cac8ee38378a1f841fec8d567eb25251821ea5f3bee5b40dd2afbe402a5ee8bfd823f007067dadc09c3c97e8950abcf4d0a568d17f241f29f83ecf8b8dc23e2ce83b,nlp-augaugmented:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4772280%2F8084649%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T090221Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D17d62f42e9c36b879cc2352bd1ad2ceb45591c87c6968f86ad84ce965bd6dade46572c4bee80f54bbb9f3d2b7aab28cbaa091b0e8edb521410a8483ab41292daf95d1bf227f4223eed2bbd03bc18e2567b678b3d43f843f36e3df4b0fba90d52b0f2a1223da76701525877593da2c149492e18741aca8ab33adecc5291208e2c3ab9e590b0a0887428147f19d1eee3e6e920213951c6efcd67811d5595ead83d1148db4edf3f0f92eae42a8bdb0f6c2385175d77f5f6e5347c082dd2c65f76d6080f67a783a8a1e71301ca53bd11d38966b79f901c7a2d21880cc555cda2454f56094db7ffae20cdb8387e7b9d195110a7eba155d97e0cb22afdc779e506d93c,nlp-aug-1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4773162%2F8133322%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T090221Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dddfe7e2bb4173720b7df95ef95ce64db7e995bafbbd5c54f17700edfe5bdbf9db3f7bd77c70fafe064a9f183e823baab0fa041f0d003ab746b1dc5a3630c37c816e73255e3b8abfe80d91b225bae0cbca791db6e0528e27b5b1e0ea02682a4790f4b2ae3fd4a1ae12c9e3338a98c9d315fdc90dc50a73d91a97994123876f1c8a0f9913ebdcb47e764ef8ac77cb2d35b30b433deeaabef21db82301100a4472d401df90bcd8389f5ef93d7d3a99f534a782c8d97e99d89cd3f14f7e7a46509dad81a6bc19f87d9aaa9376dbd5c1a4f656b1b36176c8ff5f4cdcc6258aed3c8bdeab5e19b4cbdeafaf19f7c8da0665b21a9714707af46912818a626f41cca3901'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Dataset kaggle [link](https://kaggle.com/datasets/073e4e1d58a55b6f93cd62ab184277e2ba0ac229662d9f99155c461c2deab66e)

In [ ]:
!pip3 install SetFit
!pip3 install sklearn
!pip3 install transformers
!pip3 install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from torch import cuda
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import one_hot
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from datasets import load_metric
from datasets import Dataset

import json
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import re

In [ ]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize nltk objects
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

import nltk
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /kaggle/working/corpora/wordnet.zip


replace /kaggle/working/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [ ]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove non-alphanumeric characters
    text = re.sub(r'\W', ' ', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 1]
    return ' '.join(tokens)

In [ ]:
with open('/kaggle/input/nlp-augaugmented/train_ds_augmented2.json', 'r') as f:
    data = json.load(f)

label_mapping = {
    "Politics": 0,
    "Health": 1,
    "Finance": 2,
    "Travel": 3,
    "Food": 4,
    "Education": 5,
    "Environment": 6,
    "Fashion": 7,
    "Science": 8,
    "Sports": 9,
    "Technology": 10,
    "Entertainment": 11
}

texts = []
labels = []

# Process each item
for label, sentences in data.items():
    if label in label_mapping:
        for sentence in sentences:
            #texts.append(preprocess_textsentence))
            texts.append(sentence)
            labels.append(label_mapping[label])
    else:
        print(f"Warning: Label '{label}' not found in label mapping.")

        # Create a Dataset object from the processed data
ds = Dataset.from_dict({
    "text": texts,
    "label": labels
})

# Print the dataset to verify
print(ds)

Dataset({
    features: ['text', 'label'],
    num_rows: 1254
})


In [ ]:
ds = ds.shuffle(seed=42)  # Seed for reproducibility

# Step 2: Split the dataset into train and test sets
split_ds = ds.train_test_split(test_size=0.2)  # 10% for testing, 90% for training

# Accessing the split datasets
train_dataset = split_ds['train']
test_dataset = split_ds['test']

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset
from sentence_transformers.losses import CosineSimilarityLoss

model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=1024,
    num_iterations = 40,
    num_epochs = 10
)

trainer.train()
print(trainer.evaluate())

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
/tmp/ipykernel_453/4212682733.py:7: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 80240
  Batch size = 1024
  Num epochs = 10
  Total optimization steps = 790


Step,Training Loss


{'accuracy': 0.8909090909090909}
40 int 10 epochs nlp-aug-1 train_ds_augmented.json

In [ ]:
input = ["Campus Sutra Men's Sports Jersey T-Shirt Cool-Gear: Our Proprietary Moisture Management technology. Helps to absorb and evaporate sweat quickly. Keeps you Cool & Dry. Ultra-Fresh: Fabrics treated with Ultra-Fresh Antimicrobial Technology. Ultra-Fresh is a trademark of (TRA) Inc, Ontario, Canada. Keeps you odour free."]
output = model(input)

In [ ]:
def read_texts_and_classify(file_path):
    texts = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in tqdm(lines, desc="Classifying texts", position=0, leave=True):
            text_to_classify = line.strip()
            if text_to_classify:  # Ensuring the line is not empty
                # Assuming get_prompt prepares the text for classification
                # and that you have a tokenizer and model ready for use
                texts.append(text_to_classify)
    return texts

In [ ]:
def labels_to_text(labels, label_mapping):
    # Reverse the label_mapping dictionary to map numbers to category names

    reverse_mapping = {value: key for key, value in label_mapping.items()}

    # Convert list of numbers to list of text using the reversed mapping
    text_labels = [reverse_mapping[label] for label in labels if label in reverse_mapping]

    return text_labels

In [ ]:
input_file_path = '/kaggle/input/nlp-comp/test_shuffle.txt'
# Convert to text labels
label_mapping = {
    "Politics": 0,
    "Health": 1,
    "Finance": 2,
    "Travel": 3,
    "Food": 4,
    "Education": 5,
    "Environment": 6,
    "Fashion": 7,
    "Science": 8,
    "Sports": 9,
    "Technology": 10,
    "Entertainment": 11
}
texts = read_texts_and_classify(input_file_path)
results = model(texts).cpu().numpy()
converted_labels = labels_to_text(results, label_mapping)
print(converted_labels)

In [ ]:
# Creating a DataFrame with 'label' column
df = pd.DataFrame({'Label': converted_labels})

# Writing DataFrame to a CSV file
df.to_csv('teste.csv', index_label='ID')